In [1]:
from utz import *
from njdot.data import Data, START_YEAR, END_YEAR, YEARS, YPK, PK, hist
from njdot.codes import *

In [2]:
data = Data()
data

Data(years=['2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], types=['Crash', 'Driver', 'Occupant', 'Pedestrian', 'Vehicle'], columns=None)

In [3]:
%%time
cs = data['Crash'].df(['Date', 'Total Killed', 'Total Injured', 'Pedestrians Killed', 'Pedestrians Injured'])
cs

CPU times: user 13.4 s, sys: 6.11 s, total: 19.5 s
Wall time: 18.6 s


Date  \
Year County Code Municipality Code Department Case Number                       
2001 01          01                #2001-17846            2001-12-21 18:34:00   
                                   01-00029               2001-01-01 09:30:00   
                                   01-004615              2001-04-10 14:44:00   
                                   01-004880              2001-04-15 13:56:00   
                                   01-004912              2001-04-16 10:29:00   
...                                                                       ...   
2020 21          23                B150-2020-00350A       2020-12-09 11:26:00   
                                   B150-2020-00354A       2020-12-12 18:45:00   
                                   B150-2020-00361A       2020-12-15 06:12:00   
                                   B150-2020-00369A       2020-12-19 21:37:00   
                                   B150-2020-00383A       2020-12-31 12:47:00   

                                                           Total Killed  \
Year County Code Municipality Code Department Case Number                 
2001 01          01                #2001-17846                        0   
                                   01-00029                           0   
                                   01-004615                          0   
                                   01-004880                          0   
                                   01-004912                          0   
...                                                                 ...   
2020 21          23                B150-2020-00350A                   0   
                                   B150-2020-00354A                   0   
                                   B150-2020-00361A                   1   
                                   B150-2020-00369A                   0   
                                   B150-2020-00383A                   0   

                                                           Total Injured  \
Year County Code Municipality Code Department Case Number                  
2001 01          01                #2001-17846                         0   
                                   01-00029                            0   
                                   01-004615                           4   
                                   01-004880                           1   
                                   01-004912                           0   
...                                                                  ...   
2020 21          23                B150-2020-00350A                    0   
                                   B150-2020-00354A                    0   
                                   B150-2020-00361A                    0   
                                   B150-2020-00369A                    0   
                                   B150-2020-00383A                    1   

                                                           Pedestrians Killed  \
Year County Code Municipality Code Department Case Number                       
2001 01          01                #2001-17846                              0   
                                   01-00029                                 0   
                                   01-004615                                0   
                                   01-004880                                0   
                                   01-004912                                0   
...                                                                       ...   
2020 21          23                B150-2020-00350A                         0   
                                   B150-2020-00354A                         0   
                                   B150-2020-00361A                         1   
                                   B150-2020-00369A                         0   
                                   B150-2020-00383A                         0   



In [4]:
acd = cs.Date
acd

Year  County Code  Municipality Code  Department Case Number
2001  01           01                 #2001-17846              2001-12-21 18:34:00
                                      01-00029                 2001-01-01 09:30:00
                                      01-004615                2001-04-10 14:44:00
                                      01-004880                2001-04-15 13:56:00
                                      01-004912                2001-04-16 10:29:00
                                                                       ...        
2020  21           23                 B150-2020-00350A         2020-12-09 11:26:00
                                      B150-2020-00354A         2020-12-12 18:45:00
                                      B150-2020-00361A         2020-12-15 06:12:00
                                      B150-2020-00369A         2020-12-19 21:37:00
                                      B150-2020-00383A         2020-12-31 12:47:00
Name: Date, Length: 585034

In [5]:
%%time
apc = data['Pedestrian'].df(['Is Bicyclist?', 'Physical Condition'])
apc['Physical Condition'] = apc['Physical Condition'].apply(lambda v: physical_condition[v])
apc

CPU times: user 456 ms, sys: 109 ms, total: 565 ms
Wall time: 560 ms


Is Bicyclist?  \
Year County Code Municipality Code Department Case Number                  
2001 01          01                01-007162                       False   
                                   01-11832                         True   
                                   01-13335                        False   
                                   01-16145                        False   
                                   01-7828                         False   
...                                                                  ...   
2020 21          21                2020-014845                     False   
                                   2020-016481                     False   
                 22                2020-003616                      True   
                                   2020-005705                      True   
                 23                B150-2020-00361A                False   

                                                                 Physical Condition  
Year County Code Municipality Code Department Case Number                            
2001 01          01                01-007162                 SUSPECTED MINOR INJURY  
                                   01-11832                  SUSPECTED MINOR INJURY  
                                   01-13335                SUSPECTED SERIOUS INJURY  
                                   01-16145                         POSSIBLE INJURY  
                                   01-7828                          POSSIBLE INJURY  
...                                                                             ...  
2020 21          21                2020-014845                         FATAL INJURY  
                                   2020-016481               SUSPECTED MINOR INJURY  
                 22                2020-003616                   NO APPARENT INJURY  
                                   2020-005705                      POSSIBLE INJURY  
                 23                B150-2020-00361A                    FATAL INJURY  

[172524 rows x 2 columns]

In [6]:
%%time
apcd = apc.merge(acd, left_index=True, right_index=True, how='left')

assert (apcd.isna().sum() == 0).all()

CPU times: user 17.9 s, sys: 2.4 s, total: 20.3 s
Wall time: 20.2 s


In [7]:
mh = apcd.reset_index(drop=True)
mh['Month'] = mh.Date.dt.to_period('M').astype('datetime64[M]')
mh = mh[['Month', 'Is Bicyclist?', 'Physical Condition']].value_counts().sort_index().rename('Count')
mh = (
    mh
    .reset_index()
    .pivot(
        index=['Is Bicyclist?', 'Month'],
        columns=['Physical Condition'],
        values='Count',
    )
    .fillna(0)
    .astype(int)
    .rename(columns={
        'FATAL INJURY': 'Fatality',
        'SUSPECTED SERIOUS INJURY': 'Serious Injury',
        'SUSPECTED MINOR INJURY': 'Minor Injury',
        'POSSIBLE INJURY': 'Possible Injury',
        'NO APPARENT INJURY': 'No Apparent Injury',
        'UNKNOWN': 'Unknown',
    })
)
bki2 = mh.loc[(True,)]
bk2 = bki2['Fatality'].rename('bk2')
bi2 = bki2[['Serious Injury', 'Minor Injury', 'Possible Injury' ]].sum(1).rename('bi2')
pki2 = mh.loc[(False,)]
pk2 = pki2['Fatality'].rename('pk2')
pi2 = pki2[['Serious Injury', 'Minor Injury', 'Possible Injury' ]].sum(1).rename('pi2')
pki2

Physical Condition,,Fatality,No Apparent Injury,Possible Injury,Minor Injury,Serious Injury,Unknown
Month,,,,,,,
2001-01-01,83,8,0,336,150,34,1
2001-02-01,75,11,0,230,118,19,0
2001-03-01,85,9,0,297,130,40,0
2001-04-01,93,6,0,281,171,38,0
2001-05-01,82,11,0,275,176,42,0
...,...,...,...,...,...,...,...
2020-08-01,52,9,12,84,98,37,0
2020-09-01,44,24,16,97,98,40,0
2020-10-01,65,15,17,102,114,49,0


In [8]:
bki2

Physical Condition,,Fatality,No Apparent Injury,Possible Injury,Minor Injury,Serious Injury,Unknown
Month,,,,,,,
2001-01-01,16,1,0,37,20,2,0
2001-02-01,17,1,0,34,16,3,0
2001-03-01,20,0,0,48,26,7,0
2001-04-01,51,3,0,106,65,13,1
2001-05-01,69,4,0,126,90,9,0
...,...,...,...,...,...,...,...
2020-08-01,49,1,18,79,94,18,1
2020-09-01,49,1,17,81,117,24,0
2020-10-01,39,2,13,59,77,11,0


In [9]:
pki = cs.reset_index(drop=True)
pki = (
    pki
    .assign(Month=pki.Date.dt.to_period('M').astype('datetime64'))
    [['Month', 'Pedestrians Killed', 'Pedestrians Injured']]
    .groupby('Month').sum()
)
pk1 = pki['Pedestrians Killed'].rename('pk1')
pi1 = pki['Pedestrians Injured'].rename('pi1')
pki

,Pedestrians Killed,Pedestrians Injured
Month,,
2001-01-01,8,520
2001-02-01,11,367
2001-03-01,9,471
2001-04-01,6,504
2001-05-01,11,500
...,...,...
2020-08-01,9,219
2020-09-01,24,235
2020-10-01,15,265


In [10]:
sxs(pk1, pk2, bk2)[pk1 != pk2]

,pk1,pk2,bk2
Month,,,
2004-07-01,11,9,5
2006-04-01,7,6,0
2006-05-01,12,13,1
2006-06-01,8,9,1
2006-09-01,15,16,1
2006-11-01,16,15,0
2007-09-01,7,8,1
2008-04-01,12,13,1
2008-07-01,14,13,1


In [11]:
pk1.sum(), pk2.sum(), bk2.sum()

(3145, 3160, 282)

In [12]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import qualitative
default = qualitative.Plotly
colors = [default[i] for i in [1, 3, 0, 2]]
from utz.colors import swatches
swatches(default)
swatches(colors)

<span style="font-family: monospace">#636EFA <span style="color: #636EFA">██████</span></span> <span style="font-family: monospace">#EF553B <span style="color: #EF553B">██████</span></span> <span style="font-family: monospace">#00CC96 <span style="color: #00CC96">██████</span></span> <span style="font-family: monospace">#AB63FA <span style="color: #AB63FA">██████</span></span> <span style="font-family: monospace">#FFA15A <span style="color: #FFA15A">██████</span></span> <span style="font-family: monospace">#19D3F3 <span style="color: #19D3F3">██████</span></span> <span style="font-family: monospace">#FF6692 <span style="color: #FF6692">██████</span></span> <span style="font-family: monospace">#B6E880 <span style="color: #B6E880">██████</span></span> <span style="font-family: monospace">#FF97FF <span style="color: #FF97FF">██████</span></span> <span style="font-family: monospace">#FECB52 <span style="color: #FECB52">██████</span></span>

<span style="font-family: monospace">#EF553B <span style="color: #EF553B">██████</span></span> <span style="font-family: monospace">#AB63FA <span style="color: #AB63FA">██████</span></span> <span style="font-family: monospace">#636EFA <span style="color: #636EFA">██████</span></span> <span style="font-family: monospace">#00CC96 <span style="color: #00CC96">██████</span></span>

In [13]:
spc = read_parquet('../data/crashes.pqt')
spc

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,INJURIES,STREET,FATAL_D,FATAL_P,FATAL_T,FATAL_B,dt
ACCID,,,,,,,,,,,,,,
1703,01,Atlantic,0102,Atlantic City,446,State/Interstate Authority 446 S MP 1,1.0,1.0,None,NaN,NaN,NaN,NaN,2008-01-01 00:35:00
1681,09,Hudson,0910,Union City,None,Bergenline Ave S MP 0 at 6th St,1.0,NaN,Bergenline Ave,NaN,NaN,NaN,NaN,2008-01-01 04:11:00
1659,04,Camden,0415,Gloucester Twsp,42,State Highway 42 N MP 8.2,1.0,1.0,None,NaN,NaN,NaN,NaN,2008-01-01 06:46:00
1661,20,Union,2004,Elizabeth City,624,County 624 W MP 2.2 at Ikea Dr,1.0,1.0,None,NaN,NaN,NaN,NaN,2008-01-01 12:29:00
1811,07,Essex,0716,Nutley Town,648,County 648 E MP .87 at Franklin Ave,1.0,NaN,None,NaN,NaN,NaN,NaN,2008-01-01 18:53:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12406,07,Essex,0714,Newark City,78,Interstate 78 MP 56.1,1.0,NaN,None,1.0,0.0,0.0,0.0,2023-04-12 23:17:00
12407,02,Bergen,0205,Carlstadt Boro,17,State Highway 17,1.0,NaN,None,1.0,0.0,0.0,0.0,2023-04-13 03:23:00
12408,09,Hudson,0907,Kearny Town,None,Bergen Ave,1.0,NaN,Bergen Ave,1.0,0.0,0.0,0.0,2023-04-14 02:00:00


In [47]:
cur_dt = to_dt(dt.now())
cur_yr = cur_dt.year
cur_mo = to_dt(dt.now()).to_period('M').to_timestamp()
cur_yr, cur_mo

(2023, Timestamp('2023-04-01 00:00:00'))

In [48]:
spts = spc[[f'FATAL_{t}' for t in 'DPTB' ]].groupby(spc.dt.dt.to_period('M').astype('datetime64')).sum()
sp_idx = spts.index.to_series()
spts = spts[(sp_idx.dt.year >= 2020) & (sp_idx < cur_mo)].astype(int)
sp_idx = spts.index.to_series()
sp_year = sp_idx.dt.year
spts.index.name = 'Month'
spts20 = spts[sp_year == 2020].astype(int)
spts21 = spts[sp_year >= 2021].astype(int)
spts

,FATAL_D,FATAL_P,FATAL_T,FATAL_B
Month,,,,
2020-01-01,21,8,18,2
2020-02-01,15,2,17,0
2020-03-01,11,8,16,1
2020-04-01,17,2,8,1
2020-05-01,28,9,13,2
2020-06-01,30,9,8,0
2020-07-01,30,8,19,5
2020-08-01,31,14,8,1
2020-09-01,31,5,21,0


In [49]:
api = (
    pki2
    [[
        'Fatality',
        'Serious Injury',
#         'Minor Injury',
#         'Possible Injury',
    ]]
)
api21 = spts21.FATAL_T.rename('Fatality')
api = pd.concat([ api, api21.to_frame() ])
api

,Fatality,Serious Injury
Month,,
2001-01-01,8,34.0
2001-02-01,11,19.0
2001-03-01,9,40.0
2001-04-01,6,38.0
2001-05-01,11,42.0
...,...,...
2022-11-01,17,NaN
2022-12-01,20,NaN
2023-01-01,25,NaN


In [74]:
def ksim_plot(df, title, name):
    means = df.rolling(12).mean()
    means = means.rename(columns={
        c: f'{c} (12mo avg)'
        for c in means.columns
    })
    years = list(range(int(START_YEAR), cur_yr + 1))

    fig = px.bar(
        df,
        color_discrete_sequence=colors,
        labels={'value': '', 'Month': '', 'variable': '',},
    ).update_layout(
        hovermode='x',
        plot_bgcolor="white",
        xaxis=dict(
            tickmode='array',
            tickvals=years,
            ticktext=years,
            tickangle=-45,
        ),
        legend=dict(orientation='h', x=0.5, xanchor='center'),
        title=dict(
            text=title,
            x=0.5,
        ),
    ).update_xaxes(
        gridcolor='#ccc',
    ).update_yaxes(
        gridcolor='#ccc',
    ).update_traces(
        hovertemplate='%{y:,d}',
    )
    for i, c in enumerate(means.columns):
        fig.add_trace(go.Scatter(name=c, x=means.index, y=means[c], line=dict(color=colors[i], width=3)))

    for idx, col in enumerate([ e for p in zip(df.columns, means.columns) for e in p ]):
        kwargs = dict(legendrank=100+idx)
        if idx >= 4:
            kwargs['visible'] = 'legendonly'
        fig.update_traces(
            **kwargs,
            selector=dict(name=col),
        )
    plots_dir = '../www/public/plots/njdot'
    fig.write_json(f'{plots_dir}/{name}.json')
    fig.write_image(f'{plots_dir}/{name}.png')
    return fig

In [78]:
def ksiy_plot(df, title, name):
    df = df.groupby(api.index.to_series().dt.year).sum().replace(0, nan)
    df.index.name = 'Year'
    df = df.loc[df.index < cur_yr]
    fig = px.bar(
        df,
        color_discrete_sequence=colors,
        labels={'value': '', 'Year': '', 'variable': '',},
    ).update_layout(
        hovermode='x',
        plot_bgcolor="white",
#         xaxis=dict(
#             tickmode='array',
#             tickvals=years,
#             ticktext=years,
#             tickangle=-45,
#         ),
        legend=dict(orientation='h', x=0.5, xanchor='center'),
        title=dict(
            text=title,
            x=0.5,
        ),
    ).update_xaxes(
#         gridcolor='#ccc',
    ).update_yaxes(
        gridcolor='#ccc',
    ).update_traces(
        hovertemplate='%{y:,d}',
    )

    plots_dir = '../www/public/plots/njdot'
    fig.write_json(f'{plots_dir}/{name}-by-year.json')
    fig.write_image(f'{plots_dir}/{name}-by-year.png')
    return fig

In [79]:
ksim_plot(api, 'Pedestrians killed and seriously injured (NJ)', 'ped-injuries')

In [80]:
ksiy_plot(api, 'Pedestrians killed and seriously injured (NJ)', 'ped-injuries')

In [56]:
abi = (
    bki2
    [[
        'Fatality',
        'Serious Injury',
#         'Minor Injury',
#         'Possible Injury',
    ]]
)
abi21 = spts21.FATAL_B.rename('Fatality')
abi = pd.concat([ abi, abi21.to_frame() ])
abi

,Fatality,Serious Injury
Month,,
2001-01-01,1,2.0
2001-02-01,1,3.0
2001-03-01,0,7.0
2001-04-01,3,13.0
2001-05-01,4,9.0
...,...,...
2022-11-01,0,NaN
2022-12-01,1,NaN
2023-01-01,2,NaN


In [81]:
ksim_plot(abi, 'Cyclists killed and seriously injured (NJ)', 'cyclist-injuries')

In [82]:
ksiy_plot(abi, 'Cyclists killed and seriously injured (NJ)', 'cyclist-injuries')

In [112]:
pk3 = api.loc[api.index.to_series().dt.year == 2020, 'Fatal Injury'].rename('pk3')
pk4 = spts20.FATAL_T.rename('pk4')
p_cmp = sxs(pk3, pk4)
p_cmp[pk3 != pk4]

,pk3,pk4
Month,,
2020-01-01,16,18
2020-03-01,17,16
2020-04-01,7,8
2020-05-01,12,13
2020-08-01,9,8
2020-09-01,24,21
2020-10-01,15,17
2020-12-01,13,18


In [113]:
px.bar(p_cmp, barmode='group')

In [102]:
p_cmp.sum()

pk3    173
pk4    179
dtype: int64

In [114]:
bk3 = abi.loc[abi.index.to_series().dt.year == 2020, 'Fatal Injury'].rename('bk3')
bk4 = spts20.FATAL_B.rename('bk4')
b_cmp = sxs(bk3, bk4)
b_cmp[bk3 != bk4]

,bk3,bk4
Month,,
2020-04-01,2,1
2020-07-01,3,5
2020-09-01,1,0
2020-10-01,2,3


In [106]:
px.bar(b_cmp, barmode='group')

In [107]:
b_cmp.sum()

bk3    17
bk4    18
dtype: int64